### Import Dependencies

In [1]:
import random
random.seed(0)

In [2]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms

import optuna

### Constant Variables

In [3]:
DATASET_PATH = './knee-osteoarthritis_2'

TRAIN_PATH = f'{DATASET_PATH}/train'
VAL_PATH = f'{DATASET_PATH}/val'
TEST_PATH = f'{DATASET_PATH}/test'

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

cuda


In [5]:
classes = range(4)

### Dataset Preparation

In [6]:
from src.dataset.augmented_dataset import get_KneeOsteoarthritis_Edges, KneeOsteoarthritis_Edges

transform_toTensor = transforms.Compose([transforms.ToTensor()])

train_dataset = torchvision.datasets.ImageFolder(TRAIN_PATH, transform_toTensor)
val_dataset = torchvision.datasets.ImageFolder(VAL_PATH, transform_toTensor)
test_dataset = torchvision.datasets.ImageFolder(TEST_PATH, transform_toTensor)

dataset_all = torch.utils.data.ConcatDataset([train_dataset, val_dataset, test_dataset])

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset_all, [0.7, 0.1, 0.2])

train_dataset = KneeOsteoarthritis_Edges(train_dataset)
val_dataset = KneeOsteoarthritis_Edges(val_dataset)
test_dataset = KneeOsteoarthritis_Edges(test_dataset)

In [7]:
print(len(train_dataset), len(val_dataset), len(test_dataset))

4736 676 1353


### Data Loaders

In [8]:
from torch.utils.data import DataLoader

BATCH_SIZE = 128

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

## Optuna

In [9]:
def define_model(trial, layer_size, dropout_main):
    class IntermediarySpaceModel(nn.Module):
        def __init__(self, train_device, num_classes: int = 5, dropout: float = 0.5) -> None:
            super().__init__()
            
            # Size of layer block
            
            S = layer_size
            # S = trial.suggest_int("layer_size", 12, 24)
            self.train_device = train_device
            
            # Images
            self.imagesClassifier = nn.Sequential(
                nn.Conv2d(3, S*2, kernel_size=11, stride=4, padding=2),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2),
                nn.Dropout(p=dropout*0.2),
                nn.Conv2d(S*2, S*2, kernel_size=5, padding=2),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2),
                nn.Dropout(p=dropout*0.4),
                nn.Conv2d(S*2, S*2, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.Dropout(p=dropout*0.6),
                nn.Conv2d(S*2, S, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2),
                
                nn.Flatten(),
                nn.Dropout(p=dropout*0.8),
                nn.Linear(S * 7 * 7, S*2),
            )

            self.edgesClassifier = nn.Sequential(
                nn.Conv2d(1, S*2, kernel_size=11, stride=4, padding=2),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2),
                nn.Dropout(p=dropout*0.4),
                nn.Conv2d(S*2, S*2, kernel_size=5, padding=2),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2),
                nn.Dropout(p=dropout*0.6),
                nn.Conv2d(S*2, S, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2),
                
                nn.Flatten(),
                nn.Dropout(p=dropout*0.8),
                nn.Linear(S * 6 * 6, S*2),
            )
            
            self.outputCombiner = nn.Sequential(
                nn.ReLU(inplace=True),
                nn.Dropout(p=dropout),
                nn.Linear(S*4, S*3),
                nn.ReLU(inplace=True),
                nn.Dropout(p=dropout),
                nn.Linear(S*3, S),
                nn.ReLU(inplace=True),
                nn.Dropout(p=dropout),
                nn.Linear(S, num_classes),
            )
            
        def forward(self, data: tuple) -> torch.Tensor:
            
            images, edges = data
            
            images = images.to(self.train_device)
            edges = edges.to(self.train_device)
            
            # Images
            images = self.imagesClassifier(images)
            
            # Edges
            edges = self.edgesClassifier(edges)
            
            # Combining outputs
            concated = torch.cat((images, edges), 1)
            res = self.outputCombiner(concated)
            
            return res

    # DROPOUT_MAIN = trial.suggest_float("dropout_main", 0.3, 0.7)
    model = IntermediarySpaceModel(device, 4, dropout_main)
    model = model.to(device)
    
    return model


In [10]:
import torch.optim as optim
from src.other import get_classes_frequencies
    
class_weights = get_classes_frequencies(train_dataset)
weights_tensor = torch.Tensor(list(class_weights.values())).to(device)
print(class_weights, weights_tensor)

criterion = nn.CrossEntropyLoss(weights_tensor)

{0: 2266, 1: 1530, 2: 760, 3: 180} tensor([2266., 1530.,  760.,  180.], device='cuda:0')


In [11]:
from torch.utils.tensorboard import SummaryWriter

logger = SummaryWriter(log_dir=f"logs/hyperparameter_tunning_ratio_abs")

### Objective

In [12]:
from src.training import Trainer
    
def objective(trial):
    # Hyperparameters
    layer_size = trial.suggest_int("layer_size", 8, 28)
    dropout_main = trial.suggest_float("dropout_main", 0.3, 0.7)
    initial_lr = 0.001
    lr_decay_rate_100 = trial.suggest_float("decay_rate_100", 0.3, 0.8)
    reg_type = "L2"
    reg_lambda = trial.suggest_float("reg_lambda", 0.0001, 0.1)
    
    hparams = {
      "layer_size": layer_size,
      "dropout_main": dropout_main,
      "initial_lr": initial_lr,
      "lr_decay_rate_100": lr_decay_rate_100,
      "reg_type": reg_type,
      "reg_lambda": reg_lambda,
    }
    
    # Model
    model = define_model(trial, layer_size, dropout_main)
    
    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=initial_lr)
    
    # lr scheduler
    lr_decay_rate_1 = lr_decay_rate_100**(1/100)
    lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=lr_decay_rate_1)
    
    # Trainer object
    trainer = Trainer(
      model=model,
      classes=classes,
      train_loader=train_loader,
      val_loader=val_loader,
      criterion=criterion,
      optimizer=optimizer,
      device=device,
      lr_scheduler=lr_scheduler,
      reg_type=reg_type,
      reg_lambda=0.01,
      # tensorboard_logger=logger
    )

        
    # Training loop
    MAX_STEP = 50
    differences = []
    
    tAccuracy, tLoss, vAccuracy, vLoss = 0, 0, 0, 0
    score = None
    step = 0
    for step in range(MAX_STEP):
      tAccuracy, tLoss, vAccuracy, vLoss, _ = trainer.train_single()
      
      difference = tAccuracy - vAccuracy
      differences.append(difference)
      
      # I include absolute value of valdiation accuracy and the ratio - to neither be stuck with huge overfitting (ex. tAcc=90%, vAcc=68%) 
      # nor with low accuracy of both even when training accuracy is lower than validation (ex. tAcc=33%, vAcc=35%)
      # I also applide offset to additionaly push out any low accuracies
      score = ((vAccuracy - 50) / tAccuracy) + (vAccuracy/100 - 0.5)
      
      # Early stopping mechanism was implemented - I want to stop the training whenever it starts to overfit (3 consequtive epochs with higher training accuracy than validation)
      if step > 3 and differences[step-2] > 0 and differences[step] > differences[step-1] and differences[step-1] > differences[step-2]:
        
        print(f'Training: accuracy: {tAccuracy:.3f}%, Validation: accuracy: {vAccuracy:.3f}%, Differences: {differences[-3:]}, Score: {score:.3f} Step: {step}')
        logger.add_hparams(hparams, {'hparam/tAccuracy': tAccuracy, 'hparam/vAccuracy': vAccuracy, 'score': score, 'step': step}, run_name=f"Trial_nr_{trial.number}")
        return score
      

    
    print(f'Training: accuracy: {tAccuracy:.3f}%, Validation: accuracy: {vAccuracy:.3f}%, Differences: {differences[-3:]}, Score: {score:.3f} Step: {step}')
    
    logger.add_hparams(hparams, {'hparam/tAccuracy': tAccuracy, 'hparam/vAccuracy': vAccuracy, 'score': score, 'step': step}, run_name=f"Trial_nr_{trial.number}")
    
    
    return score

In [13]:
from optuna.samplers import TPESampler

# pruner = optuna.pruners.MedianPruner()
# pruner = CustomPruner()
sampler = TPESampler()

study = optuna.create_study(directions=["maximize"], sampler=sampler)
study.optimize(objective, n_trials=50)

print("Number of finished trials: ", len(study.trials))

[I 2024-12-23 11:57:08,157] A new study created in memory with name: no-name-c9d01276-ea11-4a17-9564-1327da067735
[I 2024-12-23 11:57:22,520] Trial 0 finished with value: 0.4213391164474063 and parameters: {'layer_size': 28, 'dropout_main': 0.3804137416896371, 'decay_rate_100': 0.625538500296188, 'reg_lambda': 0.007017842271877219}. Best is trial 0 with value: 0.4213391164474063.


Training: accuracy: 80.469%, Validation: accuracy: 68.787%, Differences: [4.416005317447613, 7.352345873980482, 11.681767751479285], Score: 0.421 Step: 11


[I 2024-12-23 11:57:38,500] Trial 1 finished with value: 0.28292074769607134 and parameters: {'layer_size': 12, 'dropout_main': 0.45233424003034284, 'decay_rate_100': 0.3443304455061339, 'reg_lambda': 0.01490391195729113}. Best is trial 0 with value: 0.4213391164474063.


Training: accuracy: 70.376%, Validation: accuracy: 61.686%, Differences: [1.2513993283224067, 2.899858068127301, 8.689454062050217], Score: 0.283 Step: 13


[I 2024-12-23 11:57:57,038] Trial 2 finished with value: 0.3881902631733841 and parameters: {'layer_size': 9, 'dropout_main': 0.5036368653743739, 'decay_rate_100': 0.3978769654955977, 'reg_lambda': 0.08433172318906818}. Best is trial 0 with value: 0.4213391164474063.


...

Training: accuracy: 69.658%, Validation: accuracy: 60.355%, Differences: [1.4689199184391484, 4.3806472893011374, 9.302909603390376], Score: 0.252 Step: 12


[I 2024-12-23 12:09:23,237] Trial 48 finished with value: 0.30759354646460346 and parameters: {'layer_size': 23, 'dropout_main': 0.4918094839224133, 'decay_rate_100': 0.33940819141815637, 'reg_lambda': 0.03774892015687607}. Best is trial 40 with value: 0.4856817686400011.


Training: accuracy: 73.374%, Validation: accuracy: 63.018%, Differences: [1.6124760115144596, 6.173536702382862, 10.356403926115462], Score: 0.308 Step: 11


[I 2024-12-23 12:09:36,503] Trial 49 finished with value: 0.3361712013337523 and parameters: {'layer_size': 25, 'dropout_main': 0.5789898918762201, 'decay_rate_100': 0.6952687736396503, 'reg_lambda': 0.025147975764693742}. Best is trial 40 with value: 0.4856817686400011.


Training: accuracy: 73.142%, Validation: accuracy: 64.201%, Differences: [2.436205621301781, 3.787806852710702, 8.94070845993923], Score: 0.336 Step: 10
Number of finished trials:  50
